In [25]:
import pandas as pd
import os
import datetime
import numpy as np
import time
import smtplib
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
import dropbox
from PIL import Image 
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import *
client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')

## Get List of Files

In [5]:
def files():
    client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    metadata = client.metadata('/')
    files = [content['path'].split('/')[-1] for content in metadata['contents'] if content['is_dir'] == False]
    name = [i[0:i.find('_')] for i in files]
    files = pd.DataFrame({'file_name': files, 'c_name': name})
    dates = [content['modified'] for content in metadata['contents'] if content['is_dir'] == False]
    files['date_mod_d'] = [time.strptime(i, "%a, %d %b %Y %H:%M:%S %z") for i in dates]
    files['date_mod'] = [datetime.datetime(*files.date_mod_d.loc[i][:6]).date() for i in files.index]
    files['hour_mod'] = [datetime.datetime(*files.date_mod_d.loc[i][:6]).time() for i in files.index]
    del files['date_mod_d']
    files['check1'] = np.nan
    files['check2'] = np.nan
    return files   

In [36]:
files().head()

,file_name
0,ABA DUENAS_0001.JPG
1,ABA DUENAS_0005.JPG
2,ABA DUENAS_0009.JPG
3,ADRIANA GRAELL WK27_0001.JPG
4,ADRIANA GRAELL WK27_0013.JPG


## Get List of Today's Clients (User Input).

In [5]:
def clients():
    clients = pd.read_excel('registro.xlsx')
    clients['f_name'] = [i[0:i.find(" ")].lower() for i in clients.Name]
    clients['l_name'] = [i[i.find(" "):].lower() for i in clients.Name]
    return clients

## Final List With Emails.

In [6]:
def sendPic():
    sendPic = pd.merge(listpic(), clients(), on = ['f_name', 'l_name'], how = 'outer')
    return sendPic
                
sendPic().head()

,file_name,f_name,l_name,t_file,date,hour,size,email,Name,Email
0,ABA DUENAS_0000.JPG,aba,duenas,jpg,2017-04-21,09:53:48,0.12,NaN,Aba Duenas,fapb88ve@gmail.com
1,ABA DUENAS_0001.AVI,aba,duenas,avi,2017-04-21,09:54:36,10.82,NaN,Aba Duenas,fapb88ve@gmail.com
2,ABA DUENAS_0001.JPG,aba,duenas,jpg,2017-04-21,09:54:30,0.12,NaN,Aba Duenas,fapb88ve@gmail.com
3,ABA DUENAS_0002.JPG,aba,duenas,jpg,2017-04-21,09:54:40,0.36,NaN,Aba Duenas,fapb88ve@gmail.com
4,ABA DUENAS_0003.JPG,aba,duenas,jpg,2017-04-21,09:54:40,0.33,NaN,Aba Duenas,fapb88ve@gmail.com


## Time related Functionalities.

In [ ]:
#def time():
#    exit = datetime.datetime.now()
#    td = datetime.timedelta(minutes = 30)
#    while datetime.datetime.now() < 
#        print('early')
        
#    return datetime.datetime.now().hour
#time()

## Email Sending Functionalities

In [2]:
def SendMail4(send_list, files):
    
    msg = MIMEMultipart()
    msg['Subject'] = 'prueba'
    msg['From'] = 'fapb88ve@gmail.com'
    msg['To'] = 'fapb88ve@gmail.com'
    text = MIMEText("test")
    msg.attach(text)
    for i in files:
        if 'mp4' in i:
            part = MIMEBase('application', "octet-stream")
            fo=open('ADRIANA GRAELL WK27_0013.mpg',"rb")
            part.set_payload(fo.read() )
            Encoders.encode_base64(part)
            part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename('ADRIANA GRAELL WK27_0013.mpg'))
            msg.attach(part)
        if 'jpg' in i:
            img_data = open(k, 'rb').read()
            image = MIMEImage(img_data, name=os.path.basename(k))
            msg.attach(image)
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.login('fapb88ve@gmail.com', 'rabbitrun88ve')
    s.sendmail('fapb88ve@gmail.com', 'fapb88ve@gmail.com', msg.as_string())
    s.quit()
    
SendMail4()

TypeError: SendMail4() missing 2 required positional arguments: 'send_list' and 'files'

## Watermarking

In [8]:
def watermark(x):
    for i in x:
        if 'AVI' in i:
            clip = VideoFileClip(i)
            logo = (ImageClip("pancita4.png")
              .set_duration(clip.duration)
              .resize(height=80) # if you need to resize...
              .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
              .set_pos(("right","bottom")))
            final = CompositeVideoClip([clip, logo])
            final.write_videofile("Pancitas " + i +".mp4")
        if 'JPG' in i:
            plogo = Image.open('pancita4.png')
            plogow, plogoh = plogo.size
            plogo = plogo.resize((int(plogow*.1), int(plogoh*.1)))
            plogow, plogoh = plogo.size
            pic = Image.open(i)
            picw, pich = pic.size
            pic.paste(plogo, (picw - plogow, pich - plogoh), plogo)
            pic.save('Pancita '+ i)
            
        else:
            pass


## Download and upload files

In [1]:
def dl_files(x):
    client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    df = pd.DataFrame(x.groupby('c_name').c_name.count())
    files = x
    for i in df.index:
        c_dir = 'Pancitas Ultrasound {}'.format(i.lower().title())
        os.mkdir(c_dir)
        os.chdir('.\\' + c_dir)
        dl_df = []
        for k in files.file_name:
            if i in k:
                dl_df.append(k)
        for j in dl_df:
            f, metadata = client.get_file_and_metadata('/'+j)
            out = open(j, 'wb')
            out.write(f.read())
            out.close()
        os.chdir('..\\')    
    return 



## Code implemented

In [7]:
def full():
    client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    sup_em = input()
    
    while datetime.datetime.now() <
      

